In [3]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2025 NVIDIA Corporation
Built on Wed_Aug_20_13:58:20_Pacific_Daylight_Time_2025
Cuda compilation tools, release 13.0, V13.0.88
Build cuda_13.0.r13.0/compiler.36424714_0


Выделить на GPU массив arr из 10^9 элементов типа float и инициализировать его с помощью ядра следующим образом: arr[i] = sin((i%360)*Pi/180). Скопировать массив в память центрального процессора и посчитать ошибку err = sum_i(abs(sin((i%360)*Pi/180) - arr[i]))/10^9. Провести исследование зависимости результата от использования функций: sin, sinf, __sinf. Объяснить результат. Проверить результат при использовании массива типа double.

In [4]:
%%writefile example.cu

#include <iostream>
#include <cmath>
#include <vector>
#include <cuda_runtime.h>

#define M_PI 3.14159265358979323846

// Функции CPU

std::vector<float> float_sin_cpu(int N) {
    std::vector<float> result(N);
    for (int i = 0; i <  N; ++i) {
        float angle = (i % 360) * M_PI / 180.0f;
        result[i] = sinf(angle);
    }
    return result;
}


std::vector<double> double_sin_cpu(int N) {
    std::vector<double> result(N);
    for (int i = 0; i < N; ++i) {
        double angle = (i % 360) * M_PI / 180.0f;
        result[i] = sin(angle);
    }
    return result;
}


// CUDA Ядра

__global__ void compute_sin_float(float* d_result, int N) {
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    if (i < N) {
        float angle = (i % 360) * M_PI / 180.0f;
        d_result[i] = __sinf(angle); // __sinf возвращает float
    }
}


__global__ void compute_sin_double(double* d_result, int N) {
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    if (i < N) {
        double angle = (i % 360) * M_PI / 180.0;
        d_result[i] = sin(angle);
    }
}


// Вычисление ошибки
template<class Ty1, class Ty2>
double calculate_error(const std::vector<Ty1>& arr1, const std::vector<Ty2>& arr2)
{
	double error = 0.0;
	size_t N = arr1.size();
	for (size_t i = 0; i < N; ++i)
	{
		error += std::abs(static_cast<double>(arr1[i]) - static_cast<double>(arr2[i]));
	}
	return error / N;
}




int main() {
    const int N = 360; // 1000000000

    // CPU
    auto float_result_cpu = float_sin_cpu(N);
    auto double_result_cpu = double_sin_cpu(N);


    // GPU float
    float* d_float_result; //  хранение результатов на GPU после вычисления синуса для float
    float* h_float_result = (float*)malloc(N * sizeof(float)); // выделение памяти на CPU хосте

    cudaMalloc(&d_float_result, N * sizeof(float));

    int threadIdx_num = 256;
    int blockDim_num = (N + threadIdx_num - 1) / threadIdx_num;


    // Запуск compute_sin_float
    compute_sin_float<<<blockDim_num, threadIdx_num>>>(d_float_result, N);


    // cudaMemcpyDeviceToHost

    cudaMemcpy(h_float_result, d_float_result, N * sizeof(float), cudaMemcpyDeviceToHost);
    cudaFree(d_float_result);



    // GPU double
    double* d_double_result; //  хранение результатов на GPU полсе вычисления синуса для double
    double* h_double_result = (double*)malloc(N * sizeof(double));
    cudaMalloc(&d_double_result, N * sizeof(double));

    compute_sin_double<<<blockDim_num, threadIdx_num>>>(d_double_result, N);
    cudaMemcpy(h_double_result, d_double_result, N * sizeof(double), cudaMemcpyDeviceToHost);
    cudaFree(d_double_result);




    // float - float / double - double
    double float_error = calculate_error(float_result_cpu, std::vector<float>(h_float_result, h_float_result + N));
    double double_error = calculate_error(double_result_cpu, std::vector<double>(h_double_result, h_double_result + N));

    // float - double / double - float
    double float_double_error = calculate_error(float_result_cpu, std::vector<double>(h_double_result, h_double_result + N));
    double double_float_error = calculate_error(double_result_cpu, std::vector<float>(h_float_result, h_float_result + N));




    // общая для всех синусов
    double total_error = (float_error + double_error + float_double_error + double_float_error) / 4.0;

    std::cout << "Double Error (CPU float vs GPU float): " << float_error << std::endl;
    std::cout << "Double Error (CPU double vs GPU double): " << double_error << std::endl;
    std::cout << "Double Error (CPU float vs GPU double): " << float_double_error << std::endl;
    std::cout << "Double Error (CPU double vs GPU float): " << double_float_error << std::endl;

    free(h_float_result);
    free(h_double_result);

    return 0;
}

Overwriting example.cu


In [13]:
!nvcc example.cu -o example.exe

example.cu
tmpxft_000060b0_00000000-10_example.cudafe1.cpp


In [18]:
!./example

"." �� ���� ����७��� ��� ���譥�
��������, �ᯮ��塞�� �ணࠬ��� ��� ������ 䠩���.
